# AutoGen Põhiline Näidis

Selles koodinäidises kasutate [AutoGen](https://aka.ms/ai-agents/autogen) AI raamistikku, et luua põhivahend.

Selle näidise eesmärk on näidata samme, mida hiljem kasutame täiendavates koodinäidistes erinevate agentlike mustrite rakendamisel.


## Impordi vajalikud Python'i paketid


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Loo klient

Selles näites kasutame [GitHubi mudeleid](https://aka.ms/ai-agents-beginners/github-models), et pääseda ligi LLM-ile.

`model` on määratletud kui `gpt-4o-mini`. Proovi mudelit muuta mõneks teiseks GitHubi mudelite turul saadaval olevaks mudeliks, et näha erinevaid tulemusi.

Kiireks testiks käivitame lihtsalt lihtsa päringu - `Mis on Prantsusmaa pealinn`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Agendi määratlemine

Nüüd, kui oleme `client` seadistanud ja kinnitanud, et see töötab, loome `AssistantAgent`i. Iga agendile saab määrata:  
**name** - Lühinimi, mis on kasulik mitme agendi voogudes viitamiseks.  
**model_client** - Klient, mille lõite eelnevas etapis.  
**tools** - Saadaval olevad tööriistad, mida agent saab ülesande täitmiseks kasutada.  
**system_message** - Metasõnum, mis määratleb LLM-i ülesande, käitumise ja tooni.  

Süsteemisõnumit saab muuta, et näha, kuidas LLM reageerib. `tools` käsitleme 4. õppetunnis.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Käivita agent

Allolev funktsioon käivitab agendi. Kasutame `on_message` meetodit, et uuendada agendi olekut uue sõnumiga.

Antud juhul uuendame olekut uue kasutaja sõnumiga, mis on `"Planeeri mulle suurepärane päikeseline puhkus"`.

Saad muuta sõnumi sisu, et näha, kuidas LLM reageerib erinevalt.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
